# Exercise Predictive monitoring - Part 2
Filtering prefixes, aggregation and index-based encoding. Repeat the following tasks for the 2 event logs: small (artificial) and BPIC_2017 (real). Commit the solutions on your github repository.

1.	Import an event log as a pandas dataframe

In [1]:
import pandas as pd
import numpy as np

In [2]:
bpic = pd.read_csv('BPIC17.csv')

In [3]:
bpic

,index,Case ID,Activity,Resource,Complete Timestamp,Variant,Variant index,(case) Accepted,(case) ApplicationID,(case) CreditScore,(case) FirstWithdrawalAmount,(case) MonthlyCost,(case) NumberOfTerms,(case) OfferedAmount,(case) Selected,Action,EventID,EventOrigin,OfferID,lifecycle:transition
0,0,Offer_247135719,O_Create Offer,User_17,2016/01/02 18:17:05.720,Variant 8,8,True,Application_196483749,0,10000.0,201.76,57,10000.0,False,Created,Offer_247135719,Offer,NaN,complete
1,1,Offer_247135719,O_Created,User_17,2016/01/02 18:17:08.762,Variant 8,8,True,Application_196483749,0,10000.0,201.76,57,10000.0,False,statechange,OfferState_124849367,Offer,Offer_247135719,complete
2,2,Offer_247135719,O_Sent (online only),User_17,2016/01/02 18:19:21.330,Variant 8,8,True,Application_196483749,0,10000.0,201.76,57,10000.0,False,statechange,OfferState_440662877,Offer,Offer_247135719,complete
3,3,Offer_247135719,O_Cancelled,User_17,2016/01/02 18:21:26.034,Variant 8,8,True,Application_196483749,0,10000.0,201.76,57,10000.0,False,statechange,OfferState_591416028,Offer,Offer_247135719,complete
4,4,Offer_941964966,O_Create Offer,User_17,2016/01/02 18:21:42.022,Variant 1,1,True,Application_196483749,0,4100.0,201.76,57,10000.0,False,Created,Offer_941964966,Offer,NaN,complete
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22404,22404,Offer_1064426652,O_Create Offer,User_23,2016/02/18 22:50:01.446,Variant 2,2,True,Application_1574359134,767,6000.0,118.56,59,6000.0,True,Created,Offer_1064426652,Offer,NaN,complete
22405,22405,Offer_1064426652,O_Created,User_23,2016/02/18 22:50:02.655,Variant 2,2,True,Application_1574359134,767,6000.0,118.56,59,6000.0,True,statechange,OfferState_189916093,Offer,Offer_1064426652,complete
22406,22406,Offer_1064426652,O_Sent (mail and online),User_23,2016/02/18 22:50:22.174,Variant 2,2,True,Application_1574359134,767,6000.0,118.56,59,6000.0,True,statechange,OfferState_24747308,Offer,Offer_1064426652,complete
22407,22407,Offer_1064426652,O_Returned,User_117,2016/02/29 19:08:56.200,Variant 2,2,True,Application_1574359134,767,6000.0,118.56,59,6000.0,True,statechange,OfferState_1809485991,Offer,Offer_1064426652,complete


2.	Create an event log that retains only the attributes caseid, activity, timestamp, resource and outcome. Rename the attributes as “caseid, activity, ts, resource, outcome” if names are different (note: for BPIC17 the outcome label is “(case) Accepted”, for the small log the outcome label is “resource_anomaly_type”)

In [4]:
bpic.columns

Index(['index', 'Case ID', 'Activity', 'Resource', 'Complete Timestamp',
       'Variant', 'Variant index', '(case) Accepted', '(case) ApplicationID',
       '(case) CreditScore', '(case) FirstWithdrawalAmount',
       '(case) MonthlyCost', '(case) NumberOfTerms', '(case) OfferedAmount',
       '(case) Selected', 'Action', 'EventID', 'EventOrigin', 'OfferID',
       'lifecycle:transition'],
      dtype='object')

In [5]:
log_bpic = bpic[["Case ID","Activity", "Complete Timestamp", "Resource", "(case) Accepted"]]
log_bpic

,Case ID,Activity,Complete Timestamp,Resource,(case) Accepted
0,Offer_247135719,O_Create Offer,2016/01/02 18:17:05.720,User_17,True
1,Offer_247135719,O_Created,2016/01/02 18:17:08.762,User_17,True
2,Offer_247135719,O_Sent (online only),2016/01/02 18:19:21.330,User_17,True
3,Offer_247135719,O_Cancelled,2016/01/02 18:21:26.034,User_17,True
4,Offer_941964966,O_Create Offer,2016/01/02 18:21:42.022,User_17,True
...,...,...,...,...,...
22404,Offer_1064426652,O_Create Offer,2016/02/18 22:50:01.446,User_23,True
22405,Offer_1064426652,O_Created,2016/02/18 22:50:02.655,User_23,True
22406,Offer_1064426652,O_Sent (mail and online),2016/02/18 22:50:22.174,User_23,True
22407,Offer_1064426652,O_Returned,2016/02/29 19:08:56.200,User_117,True


In [6]:
#"Case ID","Activity", "Complete Timestamp", "Resource", "(case) Accepted"
#caseid, activity, ts, resource, outcome
log_bpic.rename(columns = {'Case ID':'caseid'}, inplace = True)
log_bpic.rename(columns = {'Activity':'activity'}, inplace = True)
log_bpic.rename(columns = {'Complete Timestamp':'ts'}, inplace = True)
log_bpic.rename(columns = {'Resource':'resource'}, inplace = True)
log_bpic.rename(columns = {'(case) Accepted':'outcome'}, inplace = True) 

C:\Users\atc98\AppData\Local\Temp\ipykernel_14152\3957972037.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  log_bpic.rename(columns = {'Case ID':'caseid'}, inplace = True)
C:\Users\atc98\AppData\Local\Temp\ipykernel_14152\3957972037.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  log_bpic.rename(columns = {'Activity':'activity'}, inplace = True)
C:\Users\atc98\AppData\Local\Temp\ipykernel_14152\3957972037.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#re

In [7]:
# 경고는 무시하면 됨
log_bpic

,caseid,activity,ts,resource,outcome
0,Offer_247135719,O_Create Offer,2016/01/02 18:17:05.720,User_17,True
1,Offer_247135719,O_Created,2016/01/02 18:17:08.762,User_17,True
2,Offer_247135719,O_Sent (online only),2016/01/02 18:19:21.330,User_17,True
3,Offer_247135719,O_Cancelled,2016/01/02 18:21:26.034,User_17,True
4,Offer_941964966,O_Create Offer,2016/01/02 18:21:42.022,User_17,True
...,...,...,...,...,...
22404,Offer_1064426652,O_Create Offer,2016/02/18 22:50:01.446,User_23,True
22405,Offer_1064426652,O_Created,2016/02/18 22:50:02.655,User_23,True
22406,Offer_1064426652,O_Sent (mail and online),2016/02/18 22:50:22.174,User_23,True
22407,Offer_1064426652,O_Returned,2016/02/29 19:08:56.200,User_117,True


In [31]:
log_bpic.to_csv("log_bpic.csv")

In [32]:
log_bpic

,caseid,activity,ts,resource,outcome
0,Offer_247135719,O_Create Offer,2016/01/02 18:17:05.720,User_17,True
1,Offer_247135719,O_Created,2016/01/02 18:17:08.762,User_17,True
2,Offer_247135719,O_Sent (online only),2016/01/02 18:19:21.330,User_17,True
3,Offer_247135719,O_Cancelled,2016/01/02 18:21:26.034,User_17,True
4,Offer_941964966,O_Create Offer,2016/01/02 18:21:42.022,User_17,True
...,...,...,...,...,...
22404,Offer_1064426652,O_Create Offer,2016/02/18 22:50:01.446,User_23,True
22405,Offer_1064426652,O_Created,2016/02/18 22:50:02.655,User_23,True
22406,Offer_1064426652,O_Sent (mail and online),2016/02/18 22:50:22.174,User_23,True
22407,Offer_1064426652,O_Returned,2016/02/29 19:08:56.200,User_117,True


3.	Create a function that does aggregation encoding of an event log:
- Activity and resource are aggregated using frequency
- Timestamp are aggregated using average

In [8]:
log_bpic['activity'].unique() # activity 종류들 ABCDE..에 해당함 8개

array(['O_Create Offer', 'O_Created', 'O_Sent (online only)',
       'O_Cancelled', 'O_Sent (mail and online)', 'O_Returned',
       'O_Accepted', 'O_Refused'], dtype=object)

In [9]:
log_bpic['resource'].unique() # resource 종류들 Alice, Bob, Chris..에 해당함 74개

array(['User_17', 'User_1', 'User_116', 'User_95', 'User_29', 'User_119',
       'User_38', 'User_68', 'User_19', 'User_90', 'User_52', 'User_117',
       'User_115', 'User_3', 'User_112', 'User_87', 'User_113', 'User_27',
       'User_99', 'User_118', 'User_8', 'User_75', 'User_120', 'User_16',
       'User_43', 'User_89', 'User_15', 'User_28', 'User_20', 'User_7',
       'User_2', 'User_30', 'User_114', 'User_14', 'User_109', 'User_9',
       'User_13', 'User_24', 'User_31', 'User_100', 'User_127', 'User_5',
       'User_10', 'User_70', 'User_22', 'User_11', 'User_49', 'User_92',
       'User_85', 'User_4', 'User_41', 'User_71', 'User_21', 'User_37',
       'User_110', 'User_44', 'User_34', 'User_74', 'User_36', 'User_18',
       'User_12', 'User_108', 'User_101', 'User_6', 'User_47', 'User_35',
       'User_50', 'User_25', 'User_102', 'User_33', 'User_144', 'User_26',
       'User_23', 'User_73', 'User_125', 'User_86', 'User_142', 'User_76',
       'User_141', 'User_32'], dtype=obje

In [10]:
# pandas pivot, reshaping <- 꼭 이 방법만 있는 건 아님

In [11]:
#data_pivot = pd.pivot_table(index='caseid', columns='activity', values='ts')
#data_pivot

In [12]:
log_bpic.loc[(log_bpic['caseid'] == 'Offer_247135719') & (log_bpic['activity'] == 'O_Create Offer')]

,caseid,activity,ts,resource,outcome
0,Offer_247135719,O_Create Offer,2016/01/02 18:17:05.720,User_17,True


In [13]:
len(log_bpic.loc[(log_bpic['caseid'] == 'Offer_247135719') & (log_bpic['activity'] == 'O_Create Offer')])

1

In [14]:
log_bpic.loc[(log_bpic['caseid'] == log_bpic['caseid'][0]) & (log_bpic['activity'] == 'O_Create Offer')]

,caseid,activity,ts,resource,outcome
0,Offer_247135719,O_Create Offer,2016/01/02 18:17:05.720,User_17,True


In [15]:
len(log_bpic.loc[(log_bpic['caseid'] == log_bpic['caseid'][0]) & (log_bpic['activity'] == 'O_Create Offer')])

1

In [16]:
# activity는 list 만들어서 자동으로 돌게끔 하면 될거 같음
# caseid는 어쩌지? -> dataframe 구조를 좀 더 공부해보자

#DataFrame 객체의 로우에 접근하려면 loc 메서드를 사용해 인덱스 값을 넘겨주면 됩니다.
#https://wikidocs.net/4368

# caseid도 집합으로 만들면 될듯? unique하게. 그래서 예를들어 (case1 & activitiy A) 이런식으로 하면 갯수가 출력되도록. 행렬.

In [17]:
len(pd.unique(log_bpic['caseid']))

5000

In [18]:
# https://wikidocs.net/4364 여기부터 보기... array는 뭐지 검색

In [19]:
list_row = pd.unique(log_bpic['caseid'])
list_row

array(['Offer_247135719', 'Offer_941964966', 'Offer_1148420274', ...,
       'Offer_1084451910', 'Offer_659154322', 'Offer_1064426652'],
      dtype=object)

In [20]:
list_col = pd.unique(log_bpic['activity'])
list_col

array(['O_Create Offer', 'O_Created', 'O_Sent (online only)',
       'O_Cancelled', 'O_Sent (mail and online)', 'O_Returned',
       'O_Accepted', 'O_Refused'], dtype=object)

In [21]:
# 비어있는 2차원 dataframe 만들기

In [22]:
empty_df = pd.DataFrame(index=list_row, columns=list_col)
empty_df

,O_Create Offer,O_Created,O_Sent (online only),O_Cancelled,O_Sent (mail and online),O_Returned,O_Accepted,O_Refused
Offer_247135719,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Offer_941964966,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Offer_1148420274,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Offer_1365106765,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Offer_1207095934,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
Offer_435530126,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Offer_144691980,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Offer_1084451910,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Offer_659154322,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
empty_df.iloc[0][0]

nan

In [24]:
empty_df.iloc[0][0] = 'something'

In [25]:
empty_df

,O_Create Offer,O_Created,O_Sent (online only),O_Cancelled,O_Sent (mail and online),O_Returned,O_Accepted,O_Refused
Offer_247135719,something,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Offer_941964966,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Offer_1148420274,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Offer_1365106765,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Offer_1207095934,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
Offer_435530126,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Offer_144691980,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Offer_1084451910,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Offer_659154322,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
# 데이터프레임을 len 으로 계산한 값으로 채우고 싶은데...

In [27]:
empty_df.iloc[0][0] = len(log_bpic.loc[(log_bpic['caseid'] == log_bpic['caseid'][0]) & (log_bpic['activity'] == log_bpic['activity'][0])])
empty_df

,O_Create Offer,O_Created,O_Sent (online only),O_Cancelled,O_Sent (mail and online),O_Returned,O_Accepted,O_Refused
Offer_247135719,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Offer_941964966,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Offer_1148420274,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Offer_1365106765,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Offer_1207095934,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
Offer_435530126,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Offer_144691980,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Offer_1084451910,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Offer_659154322,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


cnt = 1
for i in list_row:
    for j in list_col:
        print(cnt, end=' ')
        cnt += 1
    print()

# 노트북 느려져서 마크다운으로 바꿈
for i in range(0, 5000):
    for j in range(0, 8):
        empty_df.iloc[i][j] = len(log_bpic.loc[(log_bpic['caseid'] == log_bpic['caseid'][i]) & (log_bpic['activity'] == log_bpic['activity'][j])])
        print(empty_df)

empty_df

empty_df.to_csv("aggregation.csv")